In [ ]:
from transformers import T5ForConditionalGeneration as AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,BitsAndBytesConfig,pipeline,default_data_collator
from peft import get_peft_model,LoraConfig
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup

cache_dir='/var/tmp' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
model_name = 'google-t5/t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
model = AutoModel.from_pretrained(model_name,cache_dir=cache_dir)
print(model)
print(model.config)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


AttributeError: 'FloatProgress' object has no attribute 'style'

In [ ]:
x = tokenizer("summarize: i went to a club and saw a girl there and i talked to her and came back home and went to bed." , return_tensors='pt')
x

In [ ]:
with torch.no_grad():
   o =  model.generate(**x)
   print(o,o.shape)
   
   o = tokenizer.decode(o[0])
   print(o)
   
   